# lab3: This is the title of the assignment

In [8]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Imports

Authors:  
AlaaAbdulrazzaq Abdulrazzaq  
Name Author 2  
(use double space to break lines on markdowns)

In [9]:
import numpy as np
import pandas as pd 
import matplotlib as mp
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [ ]:
# Always use comments in the code to document specific steps
diabetes_Data = pd.read_csv('raw_data/diabetic_data.csv')
diabetes_Data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [11]:
admission_Data = pd.read_csv('raw_data/IDS_mapping.csv')
admission_Data.head()

,admission_type_id,description
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available


## Task 1: Cleanup the data

This section handles the cleanup of the dataset and splitting of the data

both data.csv and IDS mapping files are loaded, the different sections of the IDS mappings are sorted and taken into account when reading the file. 
Description of admission are merged into the dataset file 

In [ ]:
# Load main dataset
file_path = "/mnt/data/diabetic_data.csv"
df = pd.read_csv(file_path, dtype=str)

# Load IDS mapping file
mapping_file_path = "/mnt/data/IDS_mapping.csv"
mapping_df = pd.read_csv(mapping_file_path, dtype=str)

# Replace '?' and other non-standard missing values with NaN
df.replace(['?', 'Unknown/Invalid', 'Not Available', 'NULL'], np.nan, inplace=True)

# Convert age ranges to numeric values (mean of range)
age_map = {
    "[0-10)": 5, "[10-20)": 15, "[20-30)": 25, "[30-40)": 35,
    "[40-50)": 45, "[50-60)": 55, "[60-70)": 65, "[70-80)": 75,
    "[80-90)": 85, "[90-100)": 95
}
df["age"] = df["age"].map(age_map)

# Convert weight ranges to numeric (if available)
weight_map = {
    "[0-25)": 12.5, "[25-50)": 37.5, "[50-75)": 62.5, "[75-100)": 87.5,
    "[100-125)": 112.5, "[125-150)": 137.5, "[150-175)": 162.5, "[175-200)": 187.5
}
df["weight"] = df["weight"].map(weight_map)

# Identify breakpoints where the column names change in the mapping file
section_starts = mapping_df[mapping_df.iloc[:, 0].isna()].index.tolist()

# Extract each section dynamically
admission_type_map = mapping_df.iloc[:section_starts[0]].dropna()
discharge_disposition_map = mapping_df.iloc[section_starts[0]+1:section_starts[1]].dropna()
admission_source_map = mapping_df.iloc[section_starts[1]+1:].dropna()

# Rename columns properly
admission_type_map.columns = ["admission_type_id", "admission_type_description"]
discharge_disposition_map.columns = ["discharge_disposition_id", "discharge_disposition_description"]
admission_source_map.columns = ["admission_source_id", "admission_source_description"]

# Convert ID columns to numeric for merging
id_columns = ["admission_type_id", "discharge_disposition_id", "admission_source_id"]
for col in id_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

admission_type_map["admission_type_id"] = pd.to_numeric(admission_type_map["admission_type_id"], errors="coerce")
discharge_disposition_map["discharge_disposition_id"] = pd.to_numeric(discharge_disposition_map["discharge_disposition_id"], errors="coerce")
admission_source_map["admission_source_id"] = pd.to_numeric(admission_source_map["admission_source_id"], errors="coerce")

# Merge descriptions into the main dataset
df = df.merge(admission_type_map, on="admission_type_id", how="left")
df = df.merge(discharge_disposition_map, on="discharge_disposition_id", how="left")
df = df.merge(admission_source_map, on="admission_source_id", how="left")

# Drop original ID columns after merging
df.drop(columns=["admission_type_id", "discharge_disposition_id", "admission_source_id"], inplace=True, errors="ignore")

# Convert numeric columns properly
numeric_cols = [
    "time_in_hospital", "num_lab_procedures", "num_procedures",
    "num_medications", "number_outpatient", "number_emergency", "number_inpatient",
    "number_diagnoses", "weight"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col] = df.groupby("age")[col].transform(lambda x: x.fillna(x.mean()))  # Fill by age group average

# Handle missing values for payer_code and medical_specialty
df["payer_code"].fillna("Unknown", inplace=True)
df["medical_specialty"].fillna("Other", inplace=True)

# Fill categorical missing values with the most frequent value
categorical_cols = df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Display cleaned dataset summary
df.to_csv("cleaned_diabetic_data.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/diabetic_data.csv'

## Task 2: This is the title of task 2

This section should contain the solution of task 2.

## Results and Discussion

This section should contain:
- Results.
- Summary of best model performance:
    - Name of best model file as saved in /models.
    - Relevant scores such as: accuracy, precision, recall, F1-score, etc.
- Key discussion points.

In [ ]:
# Always use comments in the code to document specific steps